In [1]:
import os
import pandas as pd
import glob
import openpyxl
# from openpyxl import load_workbook
from openpyxl.styles import Font, NamedStyle, Alignment
from openpyxl.utils.dataframe import dataframe_to_rows
import shutil
import warnings

In [13]:
def get_excel_files(user_name: str) -> list:
    all_files = os.listdir(os.path.join("C:\\Users","Marcos","Desktop", "Meses"))
    excel_files = []
    for file in all_files:
        if file.endswith(".xlsx"):
            excel_files.append("C:/Users/Marcos/Desktop/Meses/" + file)
    return excel_files

In [14]:
def nota_de_credito(data_nueva: pd.DataFrame) -> pd.DataFrame:
    for i in range(len(data_nueva)):
        if "Nota de Crédito" in data_nueva.iloc[i,1]:
            data_nueva.iloc[i,4] = data_nueva.iloc[i,4]*-1
    return data_nueva

In [15]:
def data_cronologica(data: pd.DataFrame) -> pd.DataFrame:
    data['Fecha'] = pd.to_datetime(data['Fecha'], format='%d/%m/%Y')
    data = data.sort_values(by='Fecha', ascending=True).reset_index(drop=True)
    data['Fecha'] = data['Fecha'].dt.strftime('%d/%m/%Y') 
    return data

In [17]:
cuits = pd.read_excel('C:/Users/Marcos/Desktop/PRUEBA CUITS.xlsx')
nuevo_cliente = pd.Series([input("Ingrese el Nombre del Nuevo Cliente: ").upper(),
                           input("Ingrese el CUIT del Nuevo Cliente: ")],
                           index=cuits.columns)
cuits = cuits.append(nuevo_cliente, ignore_index=True)
cuits.to_excel('C:/Users/Marcos/Desktop/PRUEBA CUITS.xlsx', index=False)

In [2]:
file_paths = get_excel_files(1)
alldata_emitidos = pd.DataFrame()
alldata_recibidos = pd.DataFrame()


for file_path in file_paths:
    if str(cuits.iloc[(len(cuits)-1),1]) in file_path:
        if 'Emitidos' in file_path:
            alldata_emitidos = pd.concat([alldata_emitidos, pd.read_excel(file_path,skiprows=1)], axis=0, ignore_index=True).reset_index(drop=True)
        else:
            alldata_recibidos = pd.concat([alldata_recibidos, pd.read_excel(file_path,skiprows=1)], axis=0, ignore_index=True).reset_index(drop=True)  
source_file = "C:/Users/Marcos/Desktop/Oficina/Monotributo/modelo.xlsx"
excel_file_path = f"C:/Users/Marcos/Desktop/Oficina/Monotributo/{(cuits.iloc[(len(cuits)-1),0]).upper()}.xlsx"
shutil.copyfile(source_file, excel_file_path)


if len(alldata_emitidos) > 0:
    data_ventas = data_cronologica(alldata_emitidos[['Fecha', 'Tipo', 'Número Desde', 'Denominación Receptor', 'Imp. Total']])
    data_ventas = nota_de_credito(data_ventas)
    wb = openpyxl.load_workbook(excel_file_path)
    sheet = wb['VENTAS NUEVO']
    for row_idx, row_data in enumerate(data_ventas, start=7):
        for col_idx, value in enumerate(row_data, start=1):
            sheet.cell(row=row_idx, column=col_idx, value=value)
    for row in wb["VENTAS NUEVO"].iter_rows(min_row=7, max_row=len(data_ventas), min_col=1, max_col=5):
        for cell in row:
            cell.font = Font(name="Calibri", size=11) 
            if cell.column == 1:
                cell.alignment = Alignment(horizontal="right")  
            elif cell.column == 3:
                cell.alignment = Alignment(horizontal="center")  
    wb.save(excel_file_path)
if len(alldata_recibidos) > 0:
    data_compras = data_cronologica(alldata_recibidos[['Fecha', 'Tipo', 'Número Desde', 'Denominación Emisor', 'Imp. Total']])
    data_compras = nota_de_credito(data_compras)
    wb = openpyxl.load_workbook(excel_file_path)
    sheet = wb['COMPRAS NUEVO']
    for row_idx, row_data in enumerate(data_compras, start=7):
        for col_idx, value in enumerate(row_data, start=1):
            sheet.cell(row=row_idx, column=col_idx, value=value)
    for row in wb["COMPRAS NUEVO"].iter_rows(min_row=7, max_row=len(data_compras), min_col=1, max_col=5):
        for cell in row:
            cell.font = Font(name="Calibri", size=11) 
            if cell.column == 1:
                cell.alignment = Alignment(horizontal="right")  
    wb.save(excel_file_path)


NameError: name 'get_excel_files' is not defined

In [18]:
file_paths = get_excel_files(1)
alldata_emitidos = pd.DataFrame()
alldata_recibidos = pd.DataFrame()


for file_path in file_paths:
    if str(cuits.iloc[(len(cuits)-1),1]) in file_path:
        if 'Emitidos' in file_path:
            alldata_emitidos = pd.concat([alldata_emitidos, pd.read_excel(file_path,skiprows=1)], axis=0, ignore_index=True).reset_index(drop=True)
        else:
            alldata_recibidos = pd.concat([alldata_recibidos, pd.read_excel(file_path,skiprows=1)], axis=0, ignore_index=True).reset_index(drop=True)  
source_file = "C:/Users/Marcos/Desktop/Oficina/Monotributo/modelo.xlsx"
excel_file_path = f"C:/Users/Marcos/Desktop/Oficina/Monotributo/{(cuits.iloc[(len(cuits)-1),0]).upper()}.xlsx"
shutil.copyfile(source_file, excel_file_path)


if len(alldata_emitidos) > 0:
    data_ventas = data_cronologica(alldata_emitidos[['Fecha', 'Tipo', 'Número Desde', 'Denominación Receptor', 'Imp. Total']])
    data_ventas = nota_de_credito(data_ventas)
    data_excel = pd.read_excel(excel_file_path, sheet_name="VENTAS NUEVO", usecols="A:E", skiprows=5)
    size = len(data_excel)
    wb = openpyxl.load_workbook(excel_file_path)
    sheet = wb['VENTAS NUEVO']
    for row_idx, row in enumerate(openpyxl.utils.dataframe.dataframe_to_rows(data_ventas, index=False, header=False), (size+7)):
        for col_idx, value in enumerate(row, 1):
            sheet.cell(row=row_idx, column=col_idx, value=value)
    for row in wb["VENTAS NUEVO"].iter_rows(min_row=(size+7), max_row=(size+len(data_ventas)+7), min_col=1, max_col=5):
        for cell in row:
            cell.font = Font(name="Calibri", size=11) 
            if cell.column == 1:
                cell.alignment = Alignment(horizontal="right")  
            elif cell.column == 3:
                cell.alignment = Alignment(horizontal="center")  
    wb.save(excel_file_path)
if len(alldata_recibidos) > 0:
    data_compras = data_cronologica(alldata_recibidos[['Fecha', 'Tipo', 'Número Desde', 'Denominación Emisor', 'Imp. Total']])
    data_compras = nota_de_credito(data_compras)
    data_excel = pd.read_excel(excel_file_path, sheet_name="COMPRAS NUEVO", usecols="A:E", skiprows=5)
    size = len(data_excel)
    wb = openpyxl.load_workbook(excel_file_path)
    sheet = wb['COMPRAS NUEVO']
    for row_idx, row in enumerate(openpyxl.utils.dataframe.dataframe_to_rows(data_compras, index=False, header=False), (size+7)):
        for col_idx, value in enumerate(row, 1):
            sheet.cell(row=row_idx, column=col_idx, value=value)
    for row in wb["COMPRAS NUEVO"].iter_rows(min_row=(size+7), max_row=(size+len(data_compras)+7), min_col=1, max_col=5):
        for cell in row:
            cell.font = Font(name="Calibri", size=11) 
            if cell.column == 1:
                cell.alignment = Alignment(horizontal="right")  
    wb.save(excel_file_path)

C:\Users\Marcos\AppData\Local\Temp\ipykernel_16024\2376503356.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Fecha'] = pd.to_datetime(data['Fecha'], format='%d/%m/%Y')


In [3]:
import os
import pandas as pd
import warnings
import math

cuits = pd.read_excel('C:/Users/Marcos/Desktop/CUITS.xlsx')
warnings.filterwarnings("ignore")
for c in range(len(cuits)): # len(cuits)
# while c!="pene":
    # c=int(input("Valor de c: "))
    # os.system('cls')
    # input("hola: ")
    excel_filepath = f'C:/Users/Marcos/Desktop/Monotributo/{cuits.iloc[c, 0]}.xlsx'
    with pd.ExcelFile(excel_filepath) as xls:
        sheet_exists = "VENTAS NUEVO" in xls.sheet_names
    if not sheet_exists:
        # print(f"No hay hoja de ventas en el Excel de {cuits.iloc[c, 0]}\n\n")
        continue
    data_excel = pd.read_excel(excel_filepath, sheet_name="VENTAS NUEVO", usecols="A:E", skiprows=5)
    facturas = []
    factura_faltante = []
    duplicado_factura = []
    notas = []
    notas_faltante = []
    duplicado_nota = []
    data_excel['Fecha'] = pd.to_datetime(data_excel['Fecha'])
    data_excel['Fecha'] = data_excel['Fecha'].dt.strftime('%d/%m/%Y') 
    data_excel['Fecha'] = pd.to_datetime(data_excel['Fecha'], format='%d/%m/%Y')
    target_date = pd.to_datetime('2022-01-01')
    for i in range(len(data_excel)):
        data_excel_date = pd.to_datetime(data_excel.iloc[i, 0], format='%d/%m/%Y')
        if data_excel.iloc[i,0] >= target_date:
            if 'Nota' in str(data_excel.iloc[i,1]):
                if not math.isnan(data_excel.iloc[i,2]):
                    notas.append(data_excel.iloc[i,2])
            else:
                if not math.isnan(data_excel.iloc[i,2]):
                    facturas.append(data_excel.iloc[i,2])
    
    facturas.sort()
    for i in range(len(facturas)-1):
        if facturas[i]!=(int(facturas[i+1])-1):
            if facturas[i]==facturas[i+1]:
                duplicado_factura.append(facturas[i])
            else:
                factura_faltante.append(facturas[i+1]-1)
    
    notas.sort()
    for i in range(len(notas)-1):
        if notas[i]!=(int(notas[i+1])-1):
            if notas[i]==notas[i+1]:
                duplicado_nota.append(int(notas[i]))
            else:
                notas_faltante.append(int(notas[i+1]-1))
    variable=0
    if len(factura_faltante)>0:
        lista = ', '.join(map(str, factura_faltante))
        print(cuits.iloc[c,0],"| Facturas Faltantes |",lista)
        # print(facturas)
        variable=1
    if  len(duplicado_factura)>0:
        lista1 = ', '.join(map(str, duplicado_factura))
        print(cuits.iloc[c,0],"| Facturas Duplicadas |",lista1)
        variable=1
    if  len(notas_faltante)>0:
        lista = ', '.join(map(str, notas_faltante))
        print(cuits.iloc[c,0],"| Notas Faltantes |",lista)
        # print(notas)
        variable=1
    if  len(duplicado_nota)>0:
        lista1 = ', '.join(map(str, duplicado_nota))
        print(cuits.iloc[c,0],"| Notas Duplicadas |",lista1)
        variable=1
    if variable==1:
        print("\n")    

BESSON PABLO | Notas Duplicadas | 5, 6


LEIN | Facturas Faltantes | 230.0


LUCERO ALEJANDRA | Facturas Faltantes | 277.0


MELANO | Facturas Faltantes | 52, 65


MUCHIUT EDUARDO | Facturas Duplicadas | 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57


PASOTTI SANDRA | Facturas Faltantes | 153.0


ROLANDO GABRIELA | Facturas Faltantes | 664.0
ROLANDO GABRIELA | Notas Faltantes | 11
ROLANDO GABRIELA | Notas Duplicadas | 12




KeyboardInterrupt: 

In [2]:
my_list = [1,1, 7, 7, 11, 11, 4, 45,8787,9999,9999]
for i in range(len(my_list)-1):
    if my_list[i]==my_list[i+1]:
        my_list[i]=my_list[i]+1
        my_list[i+1]=my_list[i+1]+1
print(list(set(my_list)))

[2, 2, 8, 8, 12, 12, 4, 45, 8787, 10000, 10000]
